# WEB SCRAPING - Assignment 4

In [1]:
import csv
import tweepy
import webbrowser
import numpy as np
import pandas as pd
from pprint import pprint
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

## 1) 20 points

In [2]:
# Use of the Twitter developer account details given for the course
api_key ="7YSxUteYWORpCV3YvlwVqXvAB"
api_key_secret = "znK2iU94y1kH93SI3NOcDwFY4RyYcbcoIt50seH2PmpGGOzTsE"
access_token = "1411076834012418051-bLma7hywtIq8LnY1wvWZGao4lIKcVf"
access_token_secret = "3fkXkiQZlIIS59mUIgM8xj29orgivWE0TfU1wgVhF1wPe"

authorization = tweepy.OAuthHandler(api_key, api_key_secret)
authorization.set_access_token(access_token, access_token_secret)
api = tweepy.API(authorization, wait_on_rate_limit=True)

In [3]:
# Writing a client program which will extract tweets with the hash-tag '#happy'
limit = int(input("\nEnter the limit: "))

# The tweets collected would be stored as a CSV file "tweets.csv" with the following contents: id, lang, user.screen_name, text
with open("tweets.csv", "w", newline="", encoding="utf-8") as file:
    writer = csv.writer(file)
    writer.writerow(["id", "lang", "user.screen_name", "text"])

    # Handling rate-limit errors
    for tweet in tweepy.Cursor(api.search_tweets, q="#happy").items(limit):
        try:
            # Write the tweet information to the CSV file
            writer.writerow([tweet.id_str, tweet.lang, tweet.user.screen_name, tweet.text])
            
        except tweepy.TweepError as e:
            # Handle rate-limit errors
            if "Rate limit exceeded" in str(e):
                print("Rate limit exceeded. Waiting for 15 minutes...")
                time.sleep(15 * 60)
                continue
            else:
                print(f"Error: {str(e)}")
                break
    print("\nExtraction successful. 'tweets.csv' file successfully created")


Enter the limit: 10000


Rate limit reached. Sleeping for: 121
Rate limit reached. Sleeping for: 751
Rate limit reached. Sleeping for: 789
Rate limit reached. Sleeping for: 776
Rate limit reached. Sleeping for: 787



Extraction successful. 'tweets.csv' file successfully created


## 2) 60 points
A.

In [4]:
# Opening URL
driver = webdriver.Chrome()
url = "https://archive.ics.uci.edu/ml/datasets.php"
driver.get(url)

B.

In [5]:
# Printing all the content of the URL page
pprint(driver.page_source)

('<html><head>\n'
 '<title>UCI Machine Learning Repository: Data Sets</title>\n'
 '\n'
 '<!-- Stylesheet link -->\n'
 '<link rel="stylesheet" type="text/css" href="assets/ml.css">\n'
 '\n'
 '<script language="JavaScript" type="text/javascript">\n'
 '<!--\n'
 'function checkform ( form )\n'
 '{\n'
 '  // see http://www.thesitewizard.com/archive/validation.shtml\n'
 '  // for an explanation of this script and how to use it on your\n'
 '  // own website\n'
 '\n'
 '  // ** START **\n'
 '  if (form.q.value == "")\n'
 '  {\n'
 '    alert( "Please enter search terms." );\n'
 '    form.q.focus();\n'
 '    return false ;\n'
 '  }\n'
 '\n'
 '  if (getCheckedValue(form.sitesearch) == "ics.uci.edu" && '
 'form.q.value.indexOf("site:archive.ics.uci.edu/ml") == -1)\n'
 '  {\n'
 '    form.q.value = form.q.value + " site:archive.ics.uci.edu/ml";\n'
 '  }\n'
 '\n'
 '  // ** END **\n'
 '  return true ;\n'
 '}\n'
 '\n'
 '// return the value of the radio button that is checked\n'
 '// return an empty stri

C.

In [6]:
# Counting the number of times the term 'Regression' is written in the URL page
page_source = driver.page_source
count = page_source.count("Regression")
print("\nNumber of times the term 'Regression' is written in the page: " + str(count))


Number of times the term 'Regression' is written in the page: 153


D.

In [7]:
# Extracting the Name of datasets, Data Types, Default Task Attribute and the Year
option = webdriver.ChromeOptions()
option.add_argument('--headless')
driver = webdriver.Chrome(options=option)
driver.get("https://archive.ics.uci.edu/ml/datasets.php")

names = driver.find_elements(By.XPATH, "//tbody/tr/td/table/tbody/tr/td[2]/p/b/a")
data_types = driver.find_elements(By.XPATH, "//tbody/tr/td[2]/p") 
default_tasks = driver.find_elements(By.XPATH, "//tbody/tr/td[3]/p")
attribute_types = driver.find_elements(By.XPATH, "//tbody/tr/td[4]/p")
years = driver.find_elements(By.XPATH, "//tbody/tr/td[7]/p")

names = [name.text for name in names]
data_types = [data_type.text for data_type in data_types]
data_types = [data_types[data_type] for data_type in range(len(data_types)) if data_type%2]
default_tasks = [default_task.text for default_task in default_tasks]
attribute_types = [attribute_type.text for attribute_type in attribute_types]
years = [year.text for year in years]

dataframe = {'Name': names, 'Data Type': data_types[1:], 'Default Task': default_tasks[1:623], 'Attribute Types': attribute_types[1:], 'Year': years[1:]}
dataframe = pd.DataFrame(data=dataframe)

dataframe.head()

,Name,Data Type,Default Task,Attribute Types,Year
0,Abalone,Multivariate,Classification,"Categorical, Integer, Real",1995
1,Adult,Multivariate,Classification,"Categorical, Integer",1996
2,Annealing,Multivariate,Classification,"Categorical, Integer, Real",
3,Anonymous Microsoft Web Data,,Recommender-Systems,Categorical,1998
4,Arrhythmia,Multivariate,Classification,"Categorical, Integer, Real",1998


E.

In [8]:
# # Sorting the extracted data based on the year from recent to old
dataframe = dataframe.replace(r'^\s*$', np.nan, regex=True)
#dataframe['Year'] = dataframe['Year'].fillna(0)
dataframe = dataframe.dropna(subset=['Year'])
# dataframe = dataframe.astype({"Year": int})
dataframe = dataframe.sort_values(by=['Year'], ascending=False)

# Saving the sorted dataframe into a CSV file
dataframe.to_csv('Dataframe.csv')
dataframe

,Name,Data Type,Default Task,Attribute Types,Year
621,Image Recognition Task Execution Times in Mobi...,Univariate,Regression,Real,2021
593,Pedal Me Bicycle Deliveries,Time-Series,Regression,Real,2021
600,Non verbal tourists data,Multivariate,"Classification, Clustering","Integer, Real",2021
599,Gender Gap in Spanish WP,Multivariate,Classification,"Integer, Real",2021
598,Steel Industry Energy Consumption Dataset,Multivariate,Regression,NaN,2021
...,...,...,...,...,...
71,Mushroom,Multivariate,Classification,Categorical,1987
6,Audiology (Original),Multivariate,Classification,Categorical,1987
88,Soybean (Small),Multivariate,Classification,Categorical,1987
99,Thyroid Disease,"Multivariate, Domain-Theory",Classification,"Categorical, Real",1987


## 3) 10 points

__ANSWER:__

In order to crawl websites and extract structured data, 
__Scrapy__ is a Python-based web scraping framework. 
A number of features are built-in, including the ability 
to handle requests, manage cookies, and handle proxy servers.

In addition to web scraping, __Selenium__ is an automated tool that simulates 
user interactions with websites in order to automate the process. 
In addition to automating web testing, completing forms, and 
obtaining data, it can be used as a tool for data extraction.

The __Beautiful Soup__ Python library allows the parsing of HTML and XML 
documents, making it an ideal tool for web scraping. A simple interface is 
provided for traversing the parse tree and extracting the relevant information.

__Performance depends on the specific use case__, but __Scrapy__ generally offers 
better performance for large-scale, high-speed scraping tasks. 
__Beautiful Soup__ is best suited to parsing and extracting structured 
data from HTML and XML documents, while __Selenium__ is best for scenarios 
that require more complex user interactions. 

Therefore, it would not be wise to claim one of the three libraries 
as the best in this scenario (as it completely depends on the use case).

## 4) 10 points

In [9]:
# Loading the URL page with the help of webbrowser
url = "https://www.indeed.com"
webbrowser.open(url)

True